In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 102)

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
teach_df = pd.read_feather('teachers_info_clean.feather')
teach_inf_df = pd.read_feather('../datasets/suitable_teachers.feather')

In [3]:
merged_teachers = pd.merge(teach_inf_df, teach_df, left_on='teacher_id', right_on='id', how='left')

In [4]:
aggregated_data = merged_teachers.groupby('order_id').agg({
    # Уникальные идентификаторы
    'teacher_id': 'first',  
    'user_id': 'first',

    # Бинарные данные
    'enable_auto_assign': 'max',  
    'enable_assign': 'max',  
    'is_email_confirmed': 'max',
    'is_home_lessons': 'max',
    'is_external_lessons': 'max',
    'status_id.2': lambda x: x.mode()[0] if not x.mode().empty else np.nan,
    'status_display.2': lambda x: x.mode()[0] if not x.mode().empty else np.nan,
    'russian_level_id.2': 'first',
    'home_country_id.2': 'first',
    'is_confirmed.2': 'max',
    'is_display': 'max',
    'is_cell_phone_confirmed': 'max',
    'sex': lambda x: x.mode()[0] if not x.mode().empty else np.nan,
    'show_on_map': 'max',
    'send_mailing': 'max',
    'send_suitable_orders': 'max',
    'is_edited': 'max',
    'verification_status_id': 'first',
    'is_individual': 'max',
    'has_photo': 'max',

    # Числовые данные
    'lesson_duration.2': 'mean',
    'lesson_cost.2': 'mean',
    'rating_id': 'mean',
    'rating': 'mean',
    'effective_rating': 'mean',
    'area_id': 'mean',
    'amount_to_pay': 'sum',  # Используем sum вместо mean
    'is_remote_lessons': 'mean',
    'rating_for_users': 'mean',
    'rating_for_admin': 'mean',
    'orders_allowed': 'sum',  # Используем sum вместо mean
    'display_days': 'mean',
    'star_rating': 'mean',
    'rating_for_users_yesterday': 'sum',  # Используем sum вместо mean
    'review_num': 'sum',  # Используем sum вместо mean
    'teacher_age': 'mean',
    'years_teaching': 'max',  # Используем max вместо mean
    'age_at_reg': 'mean',
    'years_since_reg': 'max',  # Используем max вместо mean
    'days_since_last_visit': 'mean',
    'reg_year': 'min',
    'reg_month': 'min',
    'reg_day': 'min',
    'teaching_start_year': 'min'
}).reset_index()


In [5]:
order_df = pd.read_feather('order_cleaned.feather')

In [6]:
final_df = pd.merge(order_df, aggregated_data, left_on='id', right_on='order_id', how='left')

In [7]:
duplicates = {}

# Перебор всех колонок для сравнения
for i, col1 in enumerate(final_df.columns):
    for col2 in final_df.columns[i + 1:]:
        # Проверка на идентичность данных в колонках
        if final_df[col1].equals(final_df[col2]):
            # Запись найденных дубликатов
            duplicates.setdefault(col1, []).append(col2)

# Вывод результатов
if duplicates:
    for key, values in duplicates.items():
        print(f"Колонка '{key}' дублируется в колонках: {values}")
else:
    print("Дублирующихся колонок не найдено.")

Дублирующихся колонок не найдено.


In [8]:
final_df = final_df.drop(columns=duplicates)

In [9]:
final_df

,id,order_date,subject_id,purpose,lesson_price,lesson_duration,home_metro_id,add_info,status_id,amount_to_pay_x,planned_lesson_number,creator_id,pupil_category_new_id,lessons_per_week,minimal_price,teacher_sex,teacher_experience_from,teacher_experience_to,lesson_place_new,pupil_knowledgelvl,teacher_age_from,teacher_age_to,chosen_teachers_only,no_teachers_available,source_id,original_order_id,client_id,is_display_to_teachers,student_category,exam_preparation,num_sessions_weekly,tutor_experience_years,tutor_age_from,client_orders_count,group_avg_lesson_price,group_avg_lesson_duration,unique_clients_in_group,group_min_lesson_price,group_max_lesson_price,group_avg_lessons_per_week,date_diff_within_group,price_deviation_from_group_avg,percent_of_60min_lessons_in_group,avg_lesson_cost,experience_range,order_weekday,order_month,order_year,is_weekend,day_of_year,order_id,teacher_id,user_id,enable_auto_assign,enable_assign,is_email_confirmed,is_home_lessons,is_external_lessons,status_id.2,status_display.2,russian_level_id.2,home_country_id.2,is_confirmed.2,is_display,is_cell_phone_confirmed,sex,show_on_map,send_mailing,send_suitable_orders,is_edited,verification_status_id,is_individual,has_photo,lesson_duration.2,lesson_cost.2,rating_id,rating,effective_rating,area_id,amount_to_pay_y,is_remote_lessons,rating_for_users,rating_for_admin,orders_allowed,display_days,star_rating,rating_for_users_yesterday,review_num,teacher_age,years_teaching,age_at_reg,years_since_reg,days_since_last_visit,reg_year,reg_month,reg_day,teaching_start_year
0,2294054,2022-11-01 00:01:38.237,1.0,,0,60,NaN,None,16,0.0,0,NaN,NaN,0,0,3,0,0,0,0,0,0,0,0,18,2294000.0,675960,1,unknown,0,NaN,NaN,NaN,27,0.000000,60.0,1.0,0.0,0.0,0.000000,0.0,0.000000,100.000000,0.000000,0,1,11,2022,0,305,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2280950,2022-10-25 00:00:11.983,8.0,Разделы: высшая математика.\nДополнения: ЕГЭ.\...,1500,60,231.0,Желаемое время для занятий: Подстроюсь под реп...,16,0.0,2,NaN,15.0,4,0,2,10,0,1,0,30,90,0,0,23,NaN,1019824,1,unknown,1,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.000000,-10,1,10,2022,0,298,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1742636,2021-09-01 00:12:16.713,8.0,"Разделы: школьный курс, алгебра, алгебра логи...",1500,60,259.0,Район: м. Академическая.\nМесто проведения зан...,16,0.0,10,NaN,9.0,3,900,3,0,0,3,0,18,80,0,0,16,NaN,833270,1,unknown,0,NaN,NaN,NaN,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.000000,0,2,9,2021,0,244,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2478616,2023-02-12 10:04:36.413,10.0,Разделы: школьный курс.\nДополнения: ЕГЭ.\nКа...,1000,60,NaN,"г. Екатеринбург, время +2 МСК.\nМесто проведен...",16,0.0,10,NaN,11.0,2,0,2,3,0,4,0,25,0,0,0,23,NaN,914952,1,unknown,1,NaN,NaN,NaN,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.666667,-3,6,2,2023,1,43,2478616.0,32161.0,33272.0,1.0,0.0,0.0,1.0,1.0,3.0,1.0,5.0,7.0,1.0,1.0,0.0,2.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,58.275862,796.551724,14.586207,5.034483,0.542430,15.344828,0.0,1.000000,1139.344849,562.956909,26.0,2009.896606,4.761719,32640.50000,405.0,42.344828,37.431896,36.241379,12.479124,580.620690,2011.0,1.0,1.0,1986.0
4,2398558,2023-01-07 00:00:12.137,19.0,Разделы: вокал.\nКатегория ученика: парень 27 ...,1200,60,1630.0,Район: г. Кондрово.\nМесто проведения занятий:...,14,300.0,10,NaN,15.0,3,0,3,0,0,4,1,0,0,0,0,23,NaN,1062677,1,unknown,0,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.000000,0,5,1,2023,1,7,2398558.0,230478.0,796778.0,1.0,0.0,1.0,1.0,1.0,5.0,1.0,5.0,7.0,1.0,1.0,0.0,2.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,54.736842,821.052632,15.368421,5.7368

In [10]:
final_df.reset_index(drop=True).to_feather('final.feather')